<img src="logos/climbeco_course_logo.png" width="1000" align="left"/>
<br>
<br>
<br>
<br>
<br>

<a id='intro'></a>

# LPJ-GUESS NEE, GPP and RECO plots over Europe
This notebook is dedicated on presenting flux-maps from LPJ-GUESS output (6-hourly values). It is divided in two parts:

- [Single plot for single timepoint](#flux_map)

- [Compare plots for different timepoints](#flux_maps)


The first part includes code for reading LPJ-GUESS output files (netcdf) and presents a plot of a specific parameter (GPP, NEE or RECO) for a given timepoint. The user is able to choose which parameter is being displayed along with the exact timepoint (year, month, day and hour) through a form of dropdown widgets. The content of the plot can be updated by clicking on the RUN-button.  

The second part allows the user to select two different timepoints. Once the user has selected the timepoints and clicked the RUN-button, 3 maps (NEE, GPP and RTOT) will appear for every timepoint. This way, it is possible to perform visual inspections of all parameters at different timepoints.


<br>
<br>



## 1. Instructions on how to Run the Notebook
To execute all the code in the notebook, go to the menu at the top of the page and click on __Kernel__  and then __Restart & Run All__.

<br>
<br>

<img src="images/restart_run_all_nb_pic.png" width="250" align="center"/>

<br>
<br>

Once the notebook has finished executing, the user will be redirected to the bottom of the page. It is possible to move back to the top of the page by using the the _Back to top_ links. The user may select which part of the notebook he/she wishes to work with by using the corresponding links in the _introduction_ part.

<a id='flux_map'></a>
<br>
<br>

## 2. LPJ-GUESS single plot
This part presents a form of widgets allowing the user to select the parameter (i.e. GPP, NEE or RECO) and the exact timepoint (i.e. year, month, day and hour) for which he/she wishes to view the data for. A map will be displayed once the RUN-button is clicked. The map includes a title, a colorbar and two labels (min/max) stating the total min and total max value of the selected dataset. This is to avoid confusion with the corresponding min/max values used in the colorbar.

<br>
<br>
<div style="text-align: right"> 
    <a href="#single_plot">Go to plot</a>
</div>

In [ ]:
#Import modules:
import os
import numpy as np
import netCDF4 as cdf
from datetime import datetime
from ipywidgets import Dropdown, Button, HBox, VBox, Output
from IPython.display import display, clear_output, HTML

#Import plotting modules from cartopy:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE, LAKES

#Import plotting modules from matplotlib:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

#Make single matplotlib plots being displayed at the center of the notebook:
display(HTML("""
<style>
.output_png img {
    display: block;
    margin-left: auto;
    margin-right: auto;
}
</style>
"""))

In [ ]:
#Define paths:
path_eurocom = '/data/project/eurocom/'
path = path_eurocom+'LPJ-GUESS_2005-2018'

In [ ]:
def read_LPJ_fluxmap(filename):
    
    """
    Project:      EUROCOM
    Created:      Fri Apr 13 09:00:00 2018
    Last Changed: Fri Feb 13 10:00:00 2020
    Version:      1.1.0
    Author(s):    Ute Karstens, Karolina Pantazatou
    
    Definition:   Function that takes a string as input, signifying the path to a netCDF-datafile,
                  reads the netcdf and returns its content.
    
    Input:        1. Full path to netCDF file (data type: String)
    
    
    Output:
                  1. a list of longitudes
                  2. a list of latitudes
                  3. a list of time instances (i.e. Python datetime objects)
                  4. a 3-dimensional array with NEE values (one month time period)
                  5. a 3-dimensional array with GPP values
                  6. a 3-dimensional array with RESPIRATION values 
                  7. cell_area, flux_units, description
    """
    
    #Declare and initialize lists:
    time = [] #store sequence of timepoints
    nee = []  #store nee-arrays
    gpp = []  #store
    rtot = []
    
    #Open netcdf-file with data:
    ncfile = cdf.Dataset(filename)
            
    #print(ncfile)
    #print(ncfile.variables)
             
    lon = ncfile.variables['lons'][:]
    lat = ncfile.variables['lats'][:]
    
    cell_area = ncfile.variables['area'][:,:]
    flux_units = ncfile.variables['nee'].units
    #print(flux_units)
    
    description = ncfile.Notes
    #print(description)
            
    for m in range(ncfile.dimensions['nt'].size):
        time.append(cdf.num2date(ncfile.variables['times'][m],units=ncfile.variables['times'].units))
        nee.append(ncfile.variables['nee'][m,:,:])
        gpp.append(ncfile.variables['gpp'][m,:,:])
        rtot.append(ncfile.variables['rtot'][m,:,:])    
    
    #Close netcdf-file:
    ncfile.close()
    
    #Convert nee/gpp/rtot lists to numpy arrays:
    nee = np.array(nee,dtype='float32')
    gpp = np.array(gpp,dtype='float32')
    rtot = np.array(rtot,dtype='float32')
    
    #Return output:
    return lon, lat, time, nee, gpp, rtot, cell_area, flux_units, description

In [ ]:
def get_mesh(lon, lat):
    
    """
    Project:      EUROCOM
    Created:      Fri Apr 13 09:00:00 2018
    Last Changed: Fri Apr 13 09:00:00 2018
    Version:      1.0.0
    Author(s):    Ute Karstens
    
    Definition:   Function that takes two list of longitudes and latitudes as input and
                  and returns NumPy meshgrids of longitudes and latitudes.
    
    Input:        1. Longitudes (data type: List or NumPy array of ints or floats)
                  2. Latitudes  (data type: List or NumPy array of ints or floats)
    
    
    Output:
                  1. Meshgrid of longitudes (data type: Numpy array of ints or floats) 
                  2. Meshgrid of latitudes  (data type: Numpy array of ints or floats) 
                  
    """
    #Shift the longitudes and latitudes:
    dlat2=abs(lat[2]-lat[1])/2.
    plat=np.append((lat-dlat2),(lat[-1]+(dlat2)))
    dlon2=abs(lon[2]-lon[1])/2.
    plon=np.append((lon-dlon2),(lon[-1]+(dlon2)))
    
    #Create meshgrid:
    xx, yy = np.meshgrid(plon, plat)
    
    #Return meshgrid:
    return xx,yy

In [ ]:
def plot_map(fig, xx, yy, zz, nr, nc, iplot, title, vmin=-3, vmax=3, units='gC/m2/d', cb_name='PiYG_r'):
    
    """
    Project:      EUROCOM
    Created:      Fri Apr 13 09:00:00 2018
    Last Changed: Fri Feb 13 10:00:00 2020
    Version:      1.1.0
    Author(s):    Ute Karstens, Karolina Pantazatou
    
    Definition:   Function that creates a plot of a numpy array with labels.
    
    Input:        1. Plot object (data type: Matplotlib Figure Object)
                  2. Longitude meshgrid  (data type: List or NumPy array of ints or floats)
                  3. Latitude meshgrid  (data type: List or NumPy array of ints or floats)
                  4. 2-dimensional array (variable) to be plotted (data type: NumPy array of ints or floats)
                  5. Number of rows with plots (data type: Integer)
                  6. Number of columns with plots (data type: Integer)
                  7. Number of current plot (data type: Integer)
                  8. Plot Title (data type: String)
                  9. Colorbar min limit (data type: Integer or Float)
                 10. Colorbar max limit (data type: Integer or Float)
                 11. Variable units (data type: String)
                 12. Colorbar name (data type: String)
                 
    
    
    Output:
                  1. Map (data type: Matplotlib plot object) 
                  
    """
    
    #Set scale for features from Natural Earth:
    NEscale = '50m'    
    
    #Create a feature for Countries at 1:50m from Natural Earth:
    countries = cfeature.NaturalEarthFeature(category='cultural',
                                             name='admin_0_countries',
                                             scale=NEscale,
                                             facecolor='none')
    
    #Create a feature for Lakes at 1.50m from Natural Earth:
    lakes = cfeature.NaturalEarthFeature(category='physical',
                                         name='lakes',
                                         scale=NEscale,
                                         facecolor='none')
    
    
    #Set up a map:
    ax = fig.add_subplot(nr+1, nc, iplot, projection=ccrs.PlateCarree())
    
    #Define the spatial extent of the map (min/max lat/lon):
    img_extent = [xx.min(), xx.max(), yy.min(), yy.max()]
    
    #Set the spatial extent and the coordinate system:
    ax.set_extent(img_extent, crs=ccrs.PlateCarree())
    
    #Add gridlines:
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=0.3, color='gray', alpha=0.5)
    
    #Set gridline-parameters (spatial extent (min/max values of lat/lon) and interval(here=10))
    gl.ylocator = mticker.FixedLocator(np.arange(-90.,90.,10.))
    gl.xlocator = mticker.FixedLocator(np.arange(-180.,180.,10.))
    
    #Add Natural Earth countries:
    ax.add_feature(countries, edgecolor='black', linewidth=0.3)
    
    #Add Natural Earth lakes:
    ax.add_feature(lakes, edgecolor='black', linewidth=0.3)
    
    #Add raster with values:
    im = ax.pcolormesh(xx,yy,zz,cmap=cb_name,vmin=vmin,vmax=vmax)
    
    #Add colorbar:
    cbar=plt.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='both')
    
    #Define colorbar parameters:
    cbar.outline.set_linewidth(1.0)
    cbar.ax.tick_params(labelsize=10) 
    cbar.set_label(units,fontsize=10)
    
    #Add plot title:
    ax.set_title(title,fontsize=14)
    
    #Add explanatory text under the colorbar (raster dataset min value):
    ax.text(0.01, -0.27, 'min: %.2f' % np.nanmin(zz),
            horizontalalignment='left',transform=ax.transAxes,fontsize=11)
    
    #Add explanatory text under the colorbar (raster dataset max value):
    ax.text(0.99, -0.27, 'max: %.2f' % np.nanmax(zz),
            horizontalalignment='right',transform=ax.transAxes,fontsize=11)
    

In [ ]:
def check_dates(y, m, d):
    
    """
    Project:      ClimBEco Research School - 2020
    Created:      Fri Feb 13 10:00:00 2020
    Last Changed: Fri Feb 13 10:00:00 2020
    Version:      1.0.0
    Author(s):    Karolina Pantazatou
    
    Definition:   Function that takes 3 strings as input parameters (i.e. year, month, day)
                  controls if the date is valid and returns a boolean value.
    
    Input:        1. Year (data type: String)
                  2. Month (data type: String)
                  3. Day (data type: String)
    
    
    Output:
                  1. True/False (data type: Boolean)
    """
    
    
    #Define & initialize help variable:
    check = False
   
    #Control for February (Leap Year and normal year):
    if(((int(y)%4==0) & (int(m)==2) & (int(d)<=29)) | ((int(m)==2) & (int(d)<=28))):
        
        #Set help-variable to "true":
        check = True
    
    
    #Control for months with 31 days (Jan, Mar, May, Jul, Aug, Oct, Dec):
    elif((int(m) in [1, 3, 5, 7, 8, 10, 12]) & (int(d)<32)):
        
        #Set help-variable to "true":
        check = True
        
        
    #Control for months with 30 days (Apr, Jun, Sep, Nov):
    elif((int(m) in [4, 6, 9, 11]) & (int(d)<31)):
        
        #Set help-variable to "true":
        check = True
    
    
    #If none above the above conditions is true:
    else:
        
        #Set help-variable to "false":
        check = False
    
    
    #Return boolean variable:
    return check
        
        
    

In [ ]:
def checkpathtofile(pathtofile):
    
    """
    Project:      ClimBEco Research School - 2020
    Created:      Fri Feb 13 10:00:00 2020
    Last Changed: Fri Feb 13 10:00:00 2020
    Version:      1.0.0
    Author(s):    Karolina Pantazatou
    
    Definition:   Function that checks if a file exists in given directory.
                  It takes a string, containing the full path to a datafile, as input and 
                  controls if that filename exists. It returns a boolean value as output.
                  If the file exists, it returns "True", otherwise "False".
    
    Input:        1. Full path to datafile (data type: String)
                  
    Output:
                  1. True/False (data type: Boolean)
    """

    #Control if path to file exists:
    if(os.path.exists(pathtofile)):
        return True
    
    else:
        return False
        
        #Print error-message:
        print("\033[0;31;1m"+'File not found...'+"\033[0;31;0m\n\n")
        
        
    

In [ ]:
#Function that creates a widget-form:
def create_widget_form_single_plot():
    
    """
    Project:      ClimBEco Research School - 2020
    Created:      Fri Feb 13 11:00:00 2020
    Last Changed: Fri Feb 13 11:00:00 2020
    Version:      1.0.0
    Author(s):    Karolina Pantazatou
    
    Definition:   Function that creates a form of widgets to select the variable to plot,
                  the year, the month, the day and the time (hour) and returns a map.
    
    Input:        No input parameters
                  
    Output:       Widget form
    
    """
    
    
    #Create dropdown-widget for choice of dataset:
    data = Dropdown(options=['GPP', 'NEE', 'RTOT'],
                    value='NEE',
                    description='Data:',
                    disabled=False)

    #Create dropdown-lists for time variables (year, month, day and hour):
    years = Dropdown(options=['2015', '2016', '2017', '2018'],
                     value='2015',
                     description='Year:',
                     disabled=False)

    months = Dropdown(options=[str(i) for i in list(np.arange(1,13,1))],
                      value='1',
                      description='Month:',
                      disabled=False)

    days = Dropdown(options=[str(i) for i in list(np.arange(1,32,1))],
                    value='1',
                    description='Day:',
                    disabled=False)

    hours = Dropdown(options=['03:00', '09:00', '15:00', '21:00'],
                     value='03:00',
                     description='Hour:',
                     disabled=False)


    #Create a Button-widget to control execution:
    update_button = Button(description='Run',
                           disabled=False,
                           button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                           tooltip='Click me')

    #Add the data and time-related dropdown-widgets to a HBox:
    time_box = HBox([data, years, months, days, hours])

    #Add all widgets to a VBox:
    form = VBox([time_box, update_button])

    #Set font of all widgets in the form:
    update_button.style.button_color = '#1f78b4'


    #Initialize form output:
    form_out = Output()

    #Initialize results output:
    results_out = Output()


    #Define update function:
    def update_plot(button_c):

        #Control input:
        control_dates = check_dates(years.value, months.value, days.value)
        
        
        #Create path to file:        
        filename=path+'/lpj_'+years.value+months.value.zfill(2)+'.nc4'
        
        #Check if file exists in given path:
        control_file = checkpathtofile(filename)
        

        #If the input is correct:
        if(control_dates & control_file):
            
            
            #Get data from netcdf-file:
            lon_LPJ, lat_LPJ, time_LPJ, nee_LPJ_map, gpp_LPJ_map, rtot_LPJ_map, cell_area_LPJ, flux_units_LPJ, description_LPJ = read_LPJ_fluxmap(filename)
            
            #Get index number for selected date:
            i = [i for i in range(len(time_LPJ))
                 if time_LPJ[i]==datetime.strptime(years.value+'/'+months.value+'/'+days.value+' '+hours.value, '%Y/%m/%d %H:%M' )]
            
            #Check that data exist for given timepoint:
            if(len(i)>0):
                
                #Convert latitude/longitude lists to meshgrid:
                xx_LPJ,yy_LPJ = get_mesh(lon_LPJ,lat_LPJ)

                #Change units:
                fact = 1000.*1000.*4.*(12./44.)

                #Set unit labels:
                units_bio='gC/m2/d'
                cb_name_bio='PiYG_r'

                #Create title:
                title2=time_LPJ[i[0]].strftime("%Y-%m-%d %H:%M")
                title='LPJ-GUESS '+data.value+'   '+title2
                
                
                #Check choice of dataset:
                if(data.value=='NEE'):
                    
                    #Change units of raster dataset:
                    zz = nee_LPJ_map[i[0],:,:] * fact / cell_area_LPJ[:,:]
                   
                
                elif(data.value=='GPP'):
                    
                    #Change units of raster dataset:
                    zz = gpp_LPJ_map[i[0],:,:] * fact / cell_area_LPJ[:,:]
                    
                    
                elif(data.value=='RTOT'):
                    
                    #Change units of raster dataset:
                    zz = (gpp_LPJ_map[i[0],:,:] * fact / cell_area_LPJ[:,:])*(-1)
                    
                    
                else:
                    
                    print("\033[0;31;1m"+'Invalid dataset name...'+"\033[0;31;0m\n\n")

                
                
                #Display selection:
                with results_out:

                    #Clear previous results:
                    clear_output()
                    
                    #Create figure:
                    fig = plt.figure(figsize=(14,24))
                    
                    #Plot map:
                    plot_map(fig,xx_LPJ,yy_LPJ,zz,1,1,1,title,units=units_bio,cb_name=cb_name_bio)
                    
                    plt.show()
                    plt.close()
                    

            #No data for given timepoint: 
            else:

                #Display selection:
                with results_out:

                    #Clear previous results:
                    clear_output()

                    #Print error-message:
                    print("\033[0;31;1m"+'No data available for given timepoint...\nTry again!'+"\033[0;31;0m\n\n")


        #Invalid input values: 
        else:

            #Display selection:
            with results_out:

                #Clear previous results:
                clear_output()

                #Print error-message:
                print("\033[0;31;1m"+'Invalid date...\nTry again!'+"\033[0;31;0m\n\n")



    #Call update-function when button is clicked:
    update_button.on_click(update_plot)
    update_button.layout.margin = '50px 100px 40px 400px' #top, right, bottom, left
    time_box.layout.margin = '25px 0px 0px 0px'

    #Open form object:
    with form_out:

        #Clear previous selections in form:
        clear_output()

        #Display form and results:
        display(form, results_out)


    #Display form:
    display(form_out)  

<a id='single_plot'></a>

In [ ]:
#Call function to display form:
create_widget_form_single_plot()

<br>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>

<a id='flux_maps'></a>
<br>
<br>

## 3. LPJ-GUESS - compare plots for different timepoints
This part allows the user to select two different timepoints. Once the user has selected the timepoints and clicked the RUN-button, 3 maps (NEE, GPP and RTOT) will appear for every timepoint. This way, it is possible to perform visual inspections of all parameters at different timepoints. The maps have the same format as the ones presented in [LPJ-GUESS single plot](#flux_map).

<br>
<br>
<div style="text-align: right"> 
    <a href="#multi_plots">Go to plot</a>
</div>

In [ ]:
def plot_maps(fig, xx, yy, zz, nr, nc, iplot, title, vmin=-3, vmax=3, units='gC/m2/d', cb_name='PiYG_r'):
    
    """
    Project:      EUROCOM
    Created:      Fri Apr 13 09:00:00 2018
    Last Changed: Fri Feb 13 13:00:00 2020
    Version:      1.1.0
    Author(s):    Ute Karstens, Karolina Pantazatou
    
    Definition:   Function that creates a labeled plot (map) of a numpy array.
                  Plot parameters have been adjusted to fit a small size plot.
    
    Input:        1. Plot object (data type: Matplotlib Figure Object)
                  2. Longitude meshgrid  (data type: List or NumPy array of ints or floats)
                  3. Latitude meshgrid  (data type: List or NumPy array of ints or floats)
                  4. 2-dimensional array (variable) to be plotted (data type: NumPy array of ints or floats)
                  5. Number of rows with plots (data type: Integer)
                  6. Number of columns with plots (data type: Integer)
                  7. Number of current plot (data type: Integer)
                  8. Plot Title (data type: String)
                  9. Colorbar min limit (data type: Integer or Float)
                 10. Colorbar max limit (data type: Integer or Float)
                 11. Variable units (data type: String)
                 12. Colorbar name (data type: String)
                 
    
    
    Output:
                  1. Map (data type: Matplotlib plot object) 
                  
    """
    
    #Set scale for features from Natural Earth:
    NEscale = '50m'    
    
    #Create a feature for Countries at 1:50m from Natural Earth:
    countries = cfeature.NaturalEarthFeature(category='cultural',
                                             name='admin_0_countries',
                                             scale=NEscale,
                                             facecolor='none')
    
    #Create a feature for Lakes at 1.50m from Natural Earth:
    lakes = cfeature.NaturalEarthFeature(category='physical',
                                         name='lakes',
                                         scale=NEscale,
                                         facecolor='none')
    
    
    #Set up a map:
    ax = fig.add_subplot(nr+1, nc, iplot, projection=ccrs.PlateCarree())
    
    #Define the spatial extent of the map (min/max lat/lon):
    img_extent = [xx.min(), xx.max(), yy.min(), yy.max()]
    
    #Set the spatial extent and the coordinate system:
    ax.set_extent(img_extent, crs=ccrs.PlateCarree())
    
    #Add gridlines:
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=0.3, color='gray', alpha=0.5)
    
    #Set gridline-parameters (spatial extent (min/max values of lat/lon) and interval(here=10))
    gl.ylocator = mticker.FixedLocator(np.arange(-90.,90.,10.))
    gl.xlocator = mticker.FixedLocator(np.arange(-180.,180.,10.))
    
    #Add Natural Earth countries:
    ax.add_feature(countries, edgecolor='black', linewidth=0.3)
    
    #Add Natural Earth lakes:
    ax.add_feature(lakes, edgecolor='black', linewidth=0.3)
    
    #Add raster with values:
    im = ax.pcolormesh(xx,yy,zz,cmap=cb_name,vmin=vmin,vmax=vmax)
    
    #Add colorbar:
    cbar=plt.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='both')
    
    #Define colorbar parameters:
    cbar.outline.set_linewidth(0.5)
    cbar.ax.tick_params(labelsize=8) 
    cbar.set_label(units,fontsize=8)
    
    #Add plot title:
    ax.set_title(title,fontsize=11)
    
    #Add explanatory text under the colorbar (raster dataset min value):
    ax.text(0.01, -0.27, 'min: %.2f' % np.nanmin(zz),
            horizontalalignment='left',transform=ax.transAxes,fontsize=9)
    
    #Add explanatory text under the colorbar (raster dataset max value):
    ax.text(0.99, -0.27, 'max: %.2f' % np.nanmax(zz),
            horizontalalignment='right',transform=ax.transAxes,fontsize=9)
    

In [ ]:
#Function that creates a widget-form:
def create_widget_form_multi_maps():
    
    """
    Project:      ClimBEco Research School - 2020
    Created:      Fri Feb 13 15:00:00 2020
    Last Changed: Fri Feb 13 15:00:00 2020
    Version:      1.0.0
    Author(s):    Karolina Pantazatou
    
    Definition:   Function that creates a form of widgets to select the year,
                  the month, the day and the time (hour) for which the user
                  wishes to get the corresponding NEE, GPP and RESPIRATION 
                  maps. The user imay select 2 different timepoints and
                  visually compare the maps.
    
    Input:        No input parameters
                  
    Output:       Widget form
    
    """

    #Create dropdown-lists for time variables (year, month, day and hour) timepoint 1:
    years1 = Dropdown(options=['2015', '2016', '2017', '2018'],
                      value='2015',
                      description='Year:',
                      disabled=False)

    months1 = Dropdown(options=[str(i) for i in list(np.arange(1,13,1))],
                       value='1',
                       description='Month:',
                       disabled=False)

    days1 = Dropdown(options=[str(i) for i in list(np.arange(1,32,1))],
                     value='1',
                     description='Day:',
                     disabled=False)

    hours1 = Dropdown(options=['03:00', '09:00', '15:00', '21:00'],
                      value='03:00',
                      description='Hour:',
                      disabled=False)
    
    #Create dropdown-lists for time variables (year, month, day and hour) timepoint 2:
    years2 = Dropdown(options=['2015', '2016', '2017', '2018'],
                      value='2015',
                      description='Year:',
                      disabled=False)

    months2 = Dropdown(options=[str(i) for i in list(np.arange(1,13,1))],
                       value='1',
                       description='Month:',
                       disabled=False)

    days2 = Dropdown(options=[str(i) for i in list(np.arange(1,32,1))],
                     value='1',
                     description='Day:',
                     disabled=False)

    hours2 = Dropdown(options=['03:00', '09:00', '15:00', '21:00'],
                      value='03:00',
                      description='Hour:',
                      disabled=False)


    #Create a Button-widget to control execution:
    update_button = Button(description='Run',
                           disabled=False,
                           button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                           tooltip='Click me')
    
    
    header_time_1 = Output()
    with header_time_1:
        display(HTML('<p style="font-size:13px;font-weight:bold;color:#6495ed;">Select values for 1st timepoint: </p>'))

    header_time_2 = Output()
    with header_time_2:
        display(HTML('<p style="font-size:13px;font-weight:bold;color:#6495ed;">Select values for 2nd timepoint: </p>'))

    
    

    #Add the time-related dropdown-widgets to a HBox:
    time_hbox1 = HBox([years1, months1, days1, hours1])
    
    #Add time1-header and HBox with time-widgets to a VBox:
    time_vbox1 = VBox([header_time_1, time_hbox1])
    
    #Add the time-related dropdown-widgets to a HBox:
    time_hbox2 = HBox([years2, months2, days2, hours2])
    
    #Add time2-header and HBox with time-widgets to a VBox:
    time_vbox2 = VBox([header_time_2, time_hbox2])

    #Add all widgets to a VBox:
    form = VBox([time_vbox1, time_vbox2, update_button])

    #Set font of all widgets in the form:
    update_button.style.button_color = '#1f78b4'


    #Initialize form output:
    form_out = Output()

    #Initialize results output:
    results_out = Output()


    #Define update function:
    def update_plot(button_c):

        #Control input:
        control_dates1 = check_dates(years1.value, months1.value, days1.value)
        control_dates2 = check_dates(years2.value, months2.value, days2.value)
        
        
        #Create path to files:        
        filename1=path+'/lpj_'+years1.value+months1.value.zfill(2)+'.nc4'
        filename2=path+'/lpj_'+years2.value+months2.value.zfill(2)+'.nc4'
        
        #Check if file exists in given path:
        control_file1 = checkpathtofile(filename1)
        control_file2 = checkpathtofile(filename2)
        

        #If the input is correct:
        if(control_dates1 & control_dates2 & control_file1 & control_file2):
            
            
            #Get data from netcdf-file:
            lon_LPJ1, lat_LPJ1, time_LPJ1, nee_LPJ_map1, gpp_LPJ_map1, rtot_LPJ_map1, cell_area_LPJ1, flux_units_LPJ1, description_LPJ1 = read_LPJ_fluxmap(filename1)
            lon_LPJ2, lat_LPJ2, time_LPJ2, nee_LPJ_map2, gpp_LPJ_map2, rtot_LPJ_map2, cell_area_LPJ2, flux_units_LPJ2, description_LPJ2 = read_LPJ_fluxmap(filename2)
            
            #Get index number for selected date:
            i1 = [i for i in range(len(time_LPJ1))
                 if time_LPJ1[i]==datetime.strptime(years1.value+'/'+months1.value+'/'+days1.value+' '+hours1.value, '%Y/%m/%d %H:%M' )]
            
            #Get index number for selected date:
            i2 = [i for i in range(len(time_LPJ2))
                 if time_LPJ2[i]==datetime.strptime(years2.value+'/'+months2.value+'/'+days2.value+' '+hours2.value, '%Y/%m/%d %H:%M' )]
            
            #Check that data exist for given timepoint:
            if((len(i1)>0) & (len(i2)>0)):
                
                #Convert latitude/longitude lists to meshgrid:
                xx_LPJ1,yy_LPJ1 = get_mesh(lon_LPJ1,lat_LPJ1)
                xx_LPJ2,yy_LPJ2 = get_mesh(lon_LPJ2,lat_LPJ2)

                #Change units:
                fact = 1000.*1000.*4.*(12./44.)

                #Set unit labels:
                units_bio='gC/m2/d'
                cb_name_bio='PiYG_r'

                #Create titles:
                time_title_1=time_LPJ1[i1[0]].strftime("%Y-%m-%d %H:%M") #1st timepoint
                time_title_2=time_LPJ2[i2[0]].strftime("%Y-%m-%d %H:%M") #2nd timepoint
                
                nee_title_1='LPJ-GUESS NEE'+'   '+time_title_1 #1st timepoint
                nee_title_2='LPJ-GUESS NEE'+'   '+time_title_2 #2nd timepoint
                
                gpp_title_1='LPJ-GUESS GPP'+'   '+time_title_1 #1st timepoint
                gpp_title_2='LPJ-GUESS GPP'+'   '+time_title_2 #2nd timepoint
                
                rtot_title_1='LPJ-GUESS RTOT'+'   '+time_title_1 #1st timepoint
                rtot_title_2='LPJ-GUESS RTOT'+'   '+time_title_2 #2nd timepoint

                #Change units of raster dataset:
                zz_nee_1 = nee_LPJ_map1[i1[0],:,:] * fact / cell_area_LPJ1[:,:]
                zz_gpp_1 = gpp_LPJ_map1[i1[0],:,:] * fact / cell_area_LPJ1[:,:]
                zz_rtot_1 = (rtot_LPJ_map1[i1[0],:,:] * fact / cell_area_LPJ1[:,:])*(-1)
                zz_nee_2 = nee_LPJ_map2[i2[0],:,:] * fact / cell_area_LPJ2[:,:]
                zz_gpp_2 = gpp_LPJ_map2[i2[0],:,:] * fact / cell_area_LPJ2[:,:]
                zz_rtot_2 = (rtot_LPJ_map2[i2[0],:,:] * fact / cell_area_LPJ2[:,:])*(-1)

                
                #Display selection:
                with results_out:

                    #Clear previous results:
                    clear_output()

                    #Plot maps:
                    fig = plt.figure(figsize=(18,20))
                    plot_maps(fig,xx_LPJ1,yy_LPJ1,zz_nee_1,2,3,1,nee_title_1,units=units_bio,cb_name=cb_name_bio)
                    plot_maps(fig,xx_LPJ1,yy_LPJ1,zz_gpp_1,2,3,2,gpp_title_1,units=units_bio,cb_name=cb_name_bio)
                    plot_maps(fig,xx_LPJ1,yy_LPJ1,zz_rtot_1,2,3,3,rtot_title_1,units=units_bio,cb_name=cb_name_bio)
                    plot_maps(fig,xx_LPJ2,yy_LPJ2,zz_nee_2,2,3,4,nee_title_2,units=units_bio,cb_name=cb_name_bio)
                    plot_maps(fig,xx_LPJ2,yy_LPJ2,zz_gpp_2,2,3,5,gpp_title_2,units=units_bio,cb_name=cb_name_bio)
                    plot_maps(fig,xx_LPJ2,yy_LPJ2,zz_rtot_2,2,3,6,rtot_title_2,units=units_bio,cb_name=cb_name_bio)
                    
                    plt.show()
                    plt.close()

            #No data for given timepoint: 
            else:

                #Display selection:
                with results_out:

                    #Clear previous results:
                    clear_output()

                    #Print error-message:
                    print("\033[0;31;1m"+'No data available for given timepoint...\nTry again!'+"\033[0;31;0m\n\n")


        #Invalid input values: 
        else:

            #Display selection:
            with results_out:

                #Clear previous results:
                clear_output()

                #Print error-message:
                print("\033[0;31;1m"+'Invalid date...\nTry again!'+"\033[0;31;0m\n\n")



    #Call update-function when button is clicked:
    update_button.on_click(update_plot)
    update_button.layout.margin = '50px 100px 40px 400px' #top, right, bottom, left
    time_vbox1.layout.margin = '25px 0px 0px 0px'
    time_vbox2.layout.margin = '25px 0px 0px 0px'

    #Open form object:
    with form_out:

        #Clear previous selections in form:
        clear_output()

        #Display form and results:
        display(form, results_out)


    #Display form:
    display(form_out)  

<a id='multi_plots'></a>

In [ ]:
#Call function to display widget-form:
create_widget_form_multi_maps()

<br>
<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>

<br>
<br>
<br>
<br>
<img src="logos/climbeco_contributors_logo.png" width="1000" align="left"/>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>